In [45]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import cartopy as cart

from datetime import timedelta as delta
from datetime import datetime

from parcels import (grid, Field, FieldSet, ParticleSet, JITParticle, ScipyParticle, AdvectionRK4,
                     ErrorCode, ParticleFile, Variable, plotTrajectoriesFile)

from importlib import reload
import sys
sys.path.append('/home/students/4302001/arctic-connectivity/tools')
sys.path.append('/Users/daanreijnders/surfdrive/Thesis/repo/tools')
import plot
import lifeline

In [68]:
reload(plot)
reload(lifeline)

<module 'lifeline' from '/Users/daanreijnders/surfdrive/Thesis/repo/tools/lifeline.py'>

In [47]:
# Specify paths to velocity field and mesh
readdir_ocean = '/Users/daanreijnders/Datasets/'
readdir_ice = '/Users/daanreijnders/Datasets/'
readdir_mesh = '/Users/daanreijnders/Datasets/'

# readdir_ice = '/data/oceanparcels/input_data/CESM/0.1_deg/control/ice/arctic/'
# readdir_ocean = '/data/oceanparcels/input_data/CESM/0.1_deg/control/ocean/arctic/'
# readdir_mesh = '/scratch/DaanR/fields/'

fieldfile_ocean = 'daily_CESM_0.1degree_controlrun_year_300_arctic_region_timed.nc'
fieldfile_ice = 'monthly_icefields_CESM_0.1degree_controlrun_year_300_arctic.nc'
meshfile = 'POP_grid_coordinates.nc'

writedir = ''

In [48]:
def read_velocity_field(fieldfiles, meshfile=None):
    if not meshfile:
        meshfile = fieldfiles
    filenames = {'U': {'lon': meshfile,
                       'lat': meshfile,
                       'data':fieldfiles},
                 'V': {'lon': meshfile,
                       'lat': meshfile,
                       'data':fieldfiles}}

    variables = {'U': 'UVEL_5m',
                 'V': 'VVEL_5m'}

    dimensions = {'U': {'time': 'time',
                        'lat': 'ULAT',
                        'lon': 'ULON'},
                  'V': {'time': 'time',
                        'lat': 'ULAT',
                        'lon': 'ULON'}}
    fieldset = FieldSet.from_pop(filenames, variables, dimensions, allow_time_extrapolation=False)
    fieldset.U.vmax = 10;  fieldset.U.vmin = -10;  # set max of flow to 10 m/s
    fieldset.V.vmax = 10; fieldset.V.vmin = -10;
    return fieldset


# def add_ice_fields(fieldset, fieldfile, iceVars=['aice', 'hisnap', 'hi'], meshfile=None):
#     """Not working for now"""
#     if not meshfile:
#         meshfile = fieldfiles
#     for varName in iceVars:
#         filenames = {'lon': [meshfile],
#                      'lat': [meshfile],
#                      'data': [fieldfile]}
#         variable = (varName, varName)
#         dimensions = {'time': 'time',
#                       'lat': 'TLAT',
#                       'lon': 'TLON'}
#         field = Field.from_netcdf(filenames, variable, dimensions, allow_time_extrapolation=False)
#         fieldset.add_field(field)

In [49]:
fieldset = read_velocity_field(readdir_ocean+fieldfile_ocean, meshfile=readdir_mesh+meshfile)  

In [50]:
#add_ice_fields(fieldset, readdir_ice+fieldfile_ice, meshfile=readdir_mesh+meshfile)

In [90]:
class myGrid:
    def __init__(self, nlon, nlat, minLat=60., maxLat=90., minLon=-180, maxLon=180):
        self.nlat = nlat
        self.nlon = nlon
        self.dlat = (maxLat - minLat)/nlat
        self.dlon = 360/nlon
        
class countGrid(myGrid):
    def __init__(self, nlon, nlat, minLat=60., maxLat=90., minLon=-180, maxLon=180):
        myGrid.__init__(self, nlon, nlat, minLat, maxLat, minLon, maxLon)
        
        self.latOffset = self.dlat/2
        self.lonOffset = self.dlon/2
        self.latBounds = np.linspace(minLat, maxLat, nlat+1)
        self.lonBounds = np.linspace(minLon, maxLon, nlon+1)
        latarr = np.linspace(minLat+self.latOffset, maxLat-self.latOffset, nlat) # center
        lonarr = np.linspace(minLon+self.lonOffset, maxLon-self.lonOffset, nlon) # center
        self.lons, self.lats = np.meshgrid(lonarr, latarr)
        self.lonIdx, self.latIdx = np.meshgrid(np.arange(self.nlon), np.arange(self.nlat))
        
    def countInit(self, particleGrid):
        self.initCount = np.histogram2d(particleGrid.lonlat[:,0], particleGrid.lonlat[:,1], bins=[self.lonBounds, self.latBounds])
    
    def transMat(self):
        pass
    
class particleGrid(myGrid):
    def __init__(self, nlon, nlat, release_time, minLat=60.5, maxLat=89.5, minLon=-179.5, maxLon=179.5):
        myGrid.__init__(self, nlon, nlat, minLat, maxLat, minLon, maxLon)
        self.advected = False
        if release_time:
                self.release_time = release_time
        # Create mesh
        self.latarr = np.linspace(minLat, maxLat, nlat) # particle position
        self.lonarr = np.linspace(minLon, maxLon, nlon) # particle position
        self.lons, self.lats = np.meshgrid(self.lonarr, self.latarr)
        
        # Create pairs and flatten using a reshape
        self.lonlat_3d = np.dstack((self.lons, self.lats))
        self.initialParticleCount = self.countParticles()
        self.lonlat = np.reshape(self.lonlat_3d, (self.particleCount, 2))
        
        # Create labels
        self.lonlat_labels = np.arange(self.particleCount)
        
    def countParticles(self):
        if not hasattr(self, 'lonlat') and self.advected == False:
            self.particleCount = self.lonlat_3d.shape[0]*self.lonlat_3d.shape[1]
        else:
            self.particleCount = self.lonlat[~np.isnan(self.lonlat)].shape[0]
        return self.particleCount
            
    def getBindex(self, countGrid):
        self.bindex = np.dstack((np.searchsorted(countGrid.lonBounds, self.lonlat[:,0]), np.searchsorted(countGrid.latBounds, self.lonlat[:,1])))
        return self.bindex
# Counter does not actually track positions

In [75]:
def wrapLon(particle, fieldset, time):
    if particle.lon > 180.:
        particle.lon = particle.lon - 360.
    if particle.lon < -180.:
        particle.lon = particle.lon + 360.

def deleteParticle(particle, fieldset, time):
    particle.delete()

In [76]:
# Particle execution function
def gridAdvection(fieldset, \
                  countGrid, \
                  particleGrid, \
                  experiment_name='', \
                  runtime=delta(days=30), \
                  dt = delta(minutes=5), \
                  outputdt = delta(hours = 12)):
    pset = ParticleSet.from_list(fieldset, JITParticle, particles.lonlat[:,0], particles.lonlat[:,1])
    kernels = pset.Kernel(AdvectionRK4) + pset.Kernel(wrapLon)
    pfile = pset.ParticleFile(name = writedir+"pset_"+experiment_name, outputdt=outputdt)
    print(f"Run: Advecting particles for {runtime}")
    pset.execute(kernels, \
                 runtime = runtime, \
                 dt = dt, \
                 output_file = pfile, \
                 recovery = {ErrorCode.ErrorOutOfBounds: deleteParticle})
    return pset

In [77]:
countG = countGrid(60, 30)
particleG = particleGrid(119, 59, 0)

In [81]:
psetTest = gridAdvection(fieldset, countG, particleG, experiment_name='test')

Run: Advecting particles for 30 days, 0:00:00


INFO: Compiled JITParticleAdvectionRK4wrapLon ==> /var/folders/k_/m82s7xk97dlgd2rcy0bn6v100000gn/T/parcels-501/908de6ee054261da57a7d181c654c998.so
100% (2592000.0 of 2592000.0) |##########| Elapsed Time: 0:11:24 Time:  0:11:24


In [94]:
np.sum(np.isnan(np.array([particle.lon for particle in psetTest])))

0